<a href="https://colab.research.google.com/github/fermasia/interacciones-android/blob/main/Android_Apps_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento para Visualización
# Interacciones con apps de un smartphone (con Android)

Origen del dataset -> https://takeout.google.com/

A partir de la exportación de Mi Actividad desde Google Takeout de una cuenta de Android, en formato JSON, y preparación para visualizar en Flourish.Studio

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
# Definición de Funciones

def dismantle_time(dataframe,col):
  '''
  Fraccionar datetime en Year, Month, Day_Name y Hour
  '''
  dataframe["date"] = pd.to_datetime(dataframe[col].dt.date) # crear columna con fecha
  dataframe["year"] = dataframe[col].dt.year # año
  dataframe["month"] = dataframe[col].dt.month # mes
  dataframe['weekday'] = dataframe[col].dt.weekday # numero de día de la semana
  dataframe["day_name"] = dataframe[col].dt.day_name() # nombre de día de la semana
  dataframe['type_of_day'] = dataframe['weekday'].apply(lambda x: 'Semana_Laboral' if x < 5 else 'Fin_de_Semana') # tag weekday/weekend
  # renombrar días de la semana al español y con número antecedente para ordenar
  dataframe["day_name"] = dataframe["day_name"].replace({"Monday": "1-Lun", "Tuesday": "2-Mar","Wednesday": "\
  3-Mie","Thursday": "4-Jue","Friday": "5-Vie","Saturday": "6-Sab","Sunday": "7-Dom"})
  dataframe["hour"] = pd.to_datetime(pd.to_datetime(dataframe[col],errors = 'coerce')).dt.hour # hora
  dataframe.rename(columns = {'header':'app'}, inplace = True) # Renombrar header de la columna APPs
  return dataframe

def top(df,column,cant):
  '''
  Función para generar lista con nombres de aplicaciones según cantidad de apariciones en el dataset
  df - dataframe a procesar
  column - nombre de la columna de donde extraer los nombres
  cant - definir Top N aplicaciones
  ''' 
  return df.groupby(column)[column].count().sort_values(ascending=False).head(cant).index

def prep_takeout(url):
  '''
  Extraer del dump "Mi Actividad" de Google Takeout el las interacciones con aplicaciones
  Ajustando el Timezone a Buenos Aires, Argentina 
  '''
  takeout = pd.read_json(url) # leer el json
  takeout["time"] = pd.to_datetime(takeout["time"]).dt.tz_convert('America/Buenos_Aires') # Timezone de BsAs, Argentina
  # Llamar a funcióin para extraer fechas, horas y nombre del día, etc (dismantle_time)
  dismantle_time(takeout,"time")
  # Depurar columnas que no vamos a usar
  takeout.drop(columns=["title","titleUrl","products","description"],inplace=True)
  # Quedarnos con meses completos solam considerando el datetime actual
  takeout = takeout[takeout.date < takeout.date.max().replace(day=1)]
  return takeout

def crear_dataset(date,apps,con_horas):
  '''
  Función para crear un dataset base de aplicaciones con fechas
  para rellenar con luego con las interacciones que vengan en el JSON de Google 
  date - fecha desde la cual crear el dataset
  apps - lista o columna del dataframe con los nombres de aplicaciones
  con_horas - 1 para multiplicar por 24 las filas y agregar  una columna con horas 0-23
  '''
  df = pd.DataFrame(columns=["time","app"])
  for app in apps:
    today = pd.to_datetime('today').normalize() # fecha hasta
    if con_horas == 1:
      days = pd.DataFrame(pd.date_range(date,today,freq="60min")) # agregar 1 linea por cada hora
    else:
      days = pd.DataFrame(pd.date_range(date,today))
    days.columns = ["time"] # agregar columna dummy para mergear con el dataset de Google
    days["app"] = app # columna con el nombre de la aplicación
    df = df.append(days, ignore_index = True)
  return dismantle_time(df,"time") #llamar a la aplicación de apertura de fechas y horas en base al dataset creado

In [ ]:
## Aqui comenzamosa procesar
## El dataset en este ejemplo está subido a un repositorio github pero admite
## cualquier url que resuelva el JSON descargado de Google Takeout

# Dataset Original (url)
url = "https://raw.githubusercontent.com/fermasia/infovis/gh-pages/MiActividad_180721.json"

# Preparar Dataset (llamar a la función que preprocesa el dataset)
takeout = prep_takeout(url)

### Quedarse con las top 25 aplicaciones más usadas 
##  takeout = takeout[takeout.app.isin(top(takeout,'app',25))]
### y descartar aplicaciones core de Android  
##  takeout = takeout[~takeout.app.isin(['Pixel Launcher','OnePlus Launcher','com.ruthless.nexuslauncher','Contactos','Galaxy Buds+ Plugin'])]

# Definir manualmente el universo de aplicaciones a considerar
aplicaciones = ['WhatsApp Messenger','Google Chrome: rápido y seguro','Instagram','Facebook','Twitter','Netflix','Mercado Libre: compra fácil y rápido','Gmail','YouTube','Google Maps','Google Fotos','Calendario de Google','Cámara de Google','Spotify: Nueva música y podcasts para vos','Google Podcasts','Microsoft Teams','LinkedIn – App para Buscar Trabajo', 'PedidosYa - Delivery Online']

# Filtrar Aplicaciones
takeout = takeout[takeout.app.isin(aplicaciones)]

In [ ]:
# Ahora comienza el procesamiento para generar los datasets para Flourish.Studio
# Para los primeros partimos de una base vacía y relllenamos, para generar en cero 
# los dias y horas en los que no se haya usado el smartphone

# Crear base
base = crear_dataset('2019-01-01 00:00:00',aplicaciones,0)

# Quedarme con meses completos solamente
base = base[base.date < base.date.max().replace(day=1)]

# Agrupar por fecha y contar ocurrencias
df_med = takeout.groupby(["app","day_name","date"])['hour'].count().reset_index().rename(columns={"hour":"cuenta"})

# Joinear con la tabla base
base_medias = pd.merge(base,df_med,on=["date","app","day_name"],how="left")
base_medias["cuenta"] = base_medias["cuenta"].fillna(0)

# Renombrar Aplicaciones por cuestiones estéticas
base_medias['app'].replace({'WhatsApp Messenger':'Whatsapp','Google Chrome: rápido y seguro':'Chrome','Mercado Libre: compra fácil y rápido':'MercadoLibre','Calendario de Google':'Calendario','Cámara de Google':'Camara','Spotify: Nueva música y podcasts para vos':'Spotify','LinkedIn – App para Buscar Trabajo':'LinkedIn', 'PedidosYa - Delivery Online':'PedidosYa'},inplace=True)

# Calcular medias de cantidades de uso
df_media_diaria = base_medias.groupby(["app","date","day_name","type_of_day"]).mean().round(2).rename(columns={"cuenta":"promedio"})
df_media_diaria = df_media_diaria.reset_index()

# Prep & Dump for Flourish - Preparación del dataset para dibujar un BumpChart con el ranking de aplicaciones más usadas
df_media_diaria_mes_dia = df_media_diaria.groupby(['app','month','year']).mean('promedio')['promedio'].reset_index()
df_media_diaria_mes_dia['periodo'] = df_media_diaria_mes_dia['year'].astype(str) + "-" + df_media_diaria_mes_dia['month'].astype(str).apply(lambda x: x.zfill(2))
df_media_diaria_mes_dia.drop(columns=['month','year'],inplace=True)
df_media_diaria_bumpchart = df_media_diaria_mes_dia.pivot(index=['app'],columns='periodo',values='promedio').reset_index()
df_media_diaria_bumpchart['icon_url'] = ''
df_media_diaria_bumpchart.to_csv('df_media_diaria_bumpchart.csv',index=False)
files.download("df_media_diaria_bumpchart.csv")

# Prep & Dump for Flourish - Preparación del dataset para crear una visualización de múltiples con radar charts
df_media_diaria_facets = df_media_diaria.groupby(['app','day_name','type_of_day']).mean('promedio')['promedio'].reset_index().pivot(index=['day_name','type_of_day'],columns='app',values='promedio').reset_index()
column_order = ['day_name','type_of_day'] + df_media_diaria.groupby('app').sum('promedio')['promedio'].sort_values(ascending=False).reset_index().app.to_list()
df_media_diaria_facets = df_media_diaria_facets.reindex(column_order, axis=1)
df_media_diaria_facets.to_csv('df_media_diaria_facets.csv',index=False)
files.download("df_media_diaria_facets.csv")

# Prep & Dump for Flourish - Preparación del dataset para crear una visualización de Heatmap con horas y dias de la semana
df_heat = takeout.groupby(['hour','day_name'])['time'].count().reset_index()
df_heat.to_csv('df_heat.csv',index=False)
files.download("df_heat.csv")

# Prep & Dump for Flourish - Preparar dataset para dibujar un chart de jerarquías
# df_hieararchy = takeout.groupby(["hour","app"]).count()['time'].reset_index().rename(columns={"time":"cuenta"})
# df_hieararchy.to_csv('df_hieararchy.csv',index=False)
# files.download("df_hieararchy.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>